In [2]:
# import pandas as pd
# pd.set_option("display.max_row", 100)
# pd.set_option("display.max_column", 100)
# import numpy as np
# import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Train Data**

In [6]:
train = pd.read_csv('/content/drive/MyDrive/2021_2학기_금융/train_new.csv')

#### 크롤링 테스트 (무시)

In [18]:
import requests

In [46]:
code = '009540'
url = 'https://finance.naver.com/item/sise_day.nhn?code=' + code
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
res = requests.get(url, headers = headers)
res.encoding = 'utf-8'
res.status_code

200

In [47]:
from bs4 import BeautifulSoup
soap = BeautifulSoup(res.text, 'lxml')

In [48]:
el_table_navi = soap.find("table", class_="Nnavi")
el_td_last = el_table_navi.find("td", class_="pgRR")
pg_last = el_td_last.a.get('href').rsplit('&')[1]
pg_last = pg_last.split('=')[1]
pg_last = int(pg_last)
pg_last # 마지막 페이지 번호

545

In [52]:
import traceback
import pandas as pd

# 종목과 페이지 번호를 입력으로 받아서 일별 주가를 판다스 데이터 프레임 객체로 반환하는 함수
def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
        res = requests.get(url, headers = headers)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None

In [53]:
parse_page('004800', 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.09.15,122500.0,1000.0,122500.0,125000.0,120500.0,58216.0
2,2021.09.14,123500.0,2000.0,122500.0,125500.0,122500.0,77572.0
3,2021.09.13,121500.0,1500.0,122500.0,125500.0,120500.0,55803.0
4,2021.09.10,123000.0,2500.0,121500.0,124000.0,120500.0,93492.0
5,2021.09.09,120500.0,2000.0,119000.0,124000.0,118500.0,136179.0
9,2021.09.08,118500.0,2500.0,116000.0,120500.0,115500.0,87245.0
10,2021.09.07,116000.0,2000.0,118000.0,118000.0,115500.0,48006.0
11,2021.09.06,118000.0,1000.0,117000.0,119000.0,116500.0,26868.0
12,2021.09.03,117000.0,1000.0,118000.0,119000.0,117000.0,31674.0
13,2021.09.02,118000.0,1000.0,119000.0,119500.0,117500.0,43374.0


In [54]:
import datetime
str_datefrom = datetime.datetime.strftime(datetime.datetime(year=2020, month=6, day=30), '%Y.%m.%d')
str_datefrom

'2020.06.30'

In [67]:
code = '009540'

In [73]:
df = None
for page in range(1, pg_last+1):
    _df = parse_page(code, page)
    _df_filtered = _df[_df['날짜'] == str_datefrom]
    if df is None:
        df = _df_filtered
    else:
        df = pd.concat([df, _df_filtered])
    if len(df) == 1:
        break

In [74]:
df

,날짜,종가,전일비,시가,고가,저가,거래량
3,2020.06.30,87400.0,200.0,88600.0,89000.0,87400.0,191241.0


In [ ]:
import os
path_dir = '/content/drive/MyDrive/2021_2학기_금융/종가크롤링'
if not os.path.exists(path_dir):
    os.makedirs(path_dir)
path = os.path.join(path_dir, '2020_06_30_크롤링'))
path

In [ ]:
df.to_csv(path, index = False)

## **2020-06-30 종가 크롤링**

In [197]:
train

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d
0,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36
1,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A009540,2020-06-15,2020-11-16,110,1,6,0,3,2,5,5,6,9,한국조선해양,6,1,1,11
2,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A005160,2020-06-01,2020-07-14,31,1,6,0,3,2,5,5,6,9,동국산업,10,3,99,21
3,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,2020-05-28,2020-08-25,63,1,6,0,3,2,5,5,6,9,디와이피엔에프,10,2,99,23
4,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,2020-05-28,2020-08-13,55,1,6,0,3,2,5,5,6,9,SNT모티브,6,2,1,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31372,4006ddaa5673365096b9dd5965e20db866f6523c162f79...,A251270,2020-06-08,2020-07-02,18,1,5,99,5,2,5,3,2,5,넷마블,5,1,1,16
31373,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A026960,2019-07-01,2020-07-01,262,1,6,0,5,2,5,3,5,2,동서,7,1,1,261
31374,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A012750,2020-06-08,2020-07-29,37,1,6,0,5,2,5,3,5,2,에스원,5,1,1,16
31375,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A010130,2020-06-05,2020-07-09,24,1,6,0,5,2,5,3,5,2,고려아연,11,1,1,17


In [76]:
# 종목코드 리스트 생성
CODE_LIST = []
for i in range(len(train['종목코드'])):
  CODE = train['종목코드'][i][1:]
  CODE_LIST.append(CODE)

print(CODE_LIST)

['004800', '009540', '005160', '104460', '064960', '077970', '000880', '024060', '011150', '002310', '115450', '035720', '005930', '003620', '121800', '090350', '000880', '004560', '005290', '006800', '032680', '036540', '042510', '053450', '053980', '057540', '058630', '066570', '073110', '263800', '183490', '160980', '137950', '123040', '003670', '004170', '004770', '006400', '268600', '235980', '234340', '214870', '207940', '196170', '148150', '090430', '082270', '053800', '053300', '051910', '035420', '035720', '039130', '308170', '067630', '061970', '060260', '080720', '034730', '003220', '253840', '263600', '268600', '299900', '057540', '000240', '027740', '005930', '252670', '131370', '060250', '038680', '100130', '080520', '094360', '131370', '142210', '192080', '237690', '100130', '095610', '060900', '042670', '027740', '045970', '047810', '083790', '100700', '032500', '036200', '034940', '214320', '000270', '001510', '006050', '006360', '011150', '094940', '032620', '065950',

In [2]:
# 라이브러리 준비
import requests
from bs4 import BeautifulSoup
from time import sleep
import datetime
import traceback
import pandas as pd
import os
import csv

In [3]:
# 종목코드와 페이지 번호를 입력으로 받아서 일별 시세를 판다스 데이터 프레임 객체로 반환하는 함수
def PARSE_PAGE(CODE, PAGE):
    try:
        URL = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code = CODE, page = PAGE)
        # 네이버 금융 사이트는 반드시 헤더 정보를 요구
        HEADERS = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
        RES = requests.get(URL, headers = HEADERS)
        SOUP = BeautifulSoup(RES.text, 'lxml')
        DF = pd.read_html(str(SOUP.find("table")), header = 0)[0]
        DF = DF.dropna()
        return DF
    except Exception as e:
        traceback.print_exc()
    return None

In [4]:
# 하나의 주식에 대해 일별 시세 마지막 페이지를 확인하는 함수

def CRAWL_LAST_PAGE(CODE):
  URL = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code = CODE)
  HEADERS = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
  RES = requests.get(URL, headers = HEADERS)
  RES.encoding = 'utf-8'
  
  SOUP = BeautifulSoup(RES.text, 'lxml')

  PAGE_NAVI = SOUP.find("table", class_="Nnavi")
  TO_LAST_PAGE = PAGE_NAVI.find("td", class_="pgRR")
  
  # 네이버 금융에 아예 주식 정보가 없는 경우
  if TO_LAST_PAGE == None:
    LAST_PAGE = None
  else:
    LAST_PAGE = TO_LAST_PAGE.a.get('href').rsplit('&')[1]
    LAST_PAGE = LAST_PAGE.split('=')[1]
    LAST_PAGE = int(LAST_PAGE)
    LAST_PAGE # 마지막 페이지 번호
  
  return LAST_PAGE

In [43]:
DF_TRY = PARSE_PAGE('222390', 1)
BASE_DATE = datetime.datetime.strftime(datetime.datetime(year = 2020, month = 12, day = 30), '%Y.%m.%d')
if len(DF_TRY[DF_TRY['날짜'] >= BASE_DATE]) == 0:
  print('hello')

hello


In [45]:
# 하나의 주식에 대해 지정해준 기준일의 종가를 추출하는 함수

def CRAWL_CLOSINGS(YEAR, MONTH, DAY, LAST_PAGE, CODE):

  # 기준일 지정
  BASE_DATE = datetime.datetime.strftime(datetime.datetime(year = YEAR, month = MONTH, day = DAY), '%Y.%m.%d')
  
  # 네이버 금융에 아예 주식 정보가 없는 경우
  if LAST_PAGE == None: 
    CLOSINGS = None
  
  else:
    # 기준일 이전 데이터밖에 없는 경우
    DF_TRY = PARSE_PAGE(CODE, 1)
    if len(DF_TRY[DF_TRY['날짜'] >= BASE_DATE]) == 0:
      CLOSINGS = None

    else:
      DF_TRY['날짜']
      BASE_DF = None
      for page in range(1, LAST_PAGE + 1):               # 1 페이지부터 마지막 페이지까지 반복
        DF = PARSE_PAGE(CODE, page)                      # PARSE_PAGE(CODE, PAGE) 함수 사용
        DF_filtered = DF[DF['날짜'] == BASE_DATE]        # 기준일에 해당하는 데이터만 필터링

        if BASE_DF is None:
          BASE_DF = DF_filtered
        else:
          BASE_DF = pd.concat([BASE_DF, DF_filtered])
        if len(BASE_DF) == 1:
          break 
    
      if len(BASE_DF) == 0 :
        CLOSINGS = None
    
      else:
        CLOSINGS = int(BASE_DF['종가'])

  return CLOSINGS

In [6]:
# 하나의 주식에 대해 2020년 6월 30일의 종가를 추출하는 함수

def CRAWL_20200630_CLOSINGS(YEAR, MONTH, DAY, CODE, PAGE):
  # 기준일 지정
  BASE_DATE = datetime.datetime.strftime(datetime.datetime(year = YEAR, month = MONTH, day = DAY), '%Y.%m.%d')
  
  DF = PARSE_PAGE(CODE, PAGE)                      # PARSE_PAGE(CODE, PAGE) 함수 사용
  BASE_DF = DF[DF['날짜'] == BASE_DATE]            # 기준일에 해당하는 데이터만 필터링
  
  # 6월 30일에 해당하는 데이터가 없는 경우 (→ 후처리 해줄 것)
  if len(BASE_DF) == 0:
    CLOSINGS = None
  # 추출된 종가  
  else:
    CLOSINGS = int(BASE_DF['종가'])
  
  return CLOSINGS

In [127]:
# 2020년 6월 30일 종가 추출

CLOSINGS_LIST = []
for code in CODE_LIST:
  CODE = code
  PAGE = 31
  CLOSINGS = CRAWL_20200630_CLOSINGS(2020,6,30,CODE, PAGE)
  CLOSINGS_LIST.append(CLOSINGS)

CLOSINGS_LIST

[62800,
 87400,
 2540,
 9420,
 37900,
 6040,
 22150,
 6780,
 3825,
 29900,
 24550,
 267500,
 52800,
 3220,
 6130,
 7090,
 22150,
 7120,
 28150,
 6670,
 2145,
 5010,
 3630,
 4935,
 9050,
 2010,
 4340,
 63200,
 10000,
 5590,
 92600,
 12900,
 5150,
 5030,
 75100,
 220500,
 2985,
 363500,
 96100,
 53500,
 26800,
 13100,
 775000,
 266500,
 24800,
 167500,
 19300,
 49900,
 5940,
 490500,
 267000,
 267500,
 38350,
 30150,
 18600,
 8100,
 1340,
 17900,
 291000,
 17200,
 24950,
 8080,
 96100,
 4680,
 2010,
 12950,
 927,
 52800,
 5335,
 7460,
 49350,
 6020,
 5400,
 4000,
 8340,
 7460,
 6370,
 79800,
 40300,
 5400,
 21400,
 1195,
 7250,
 927,
 9120,
 23750,
 15950,
 3160,
 64000,
 5900,
 4665,
 44700,
 32050,
 876,
 2555,
 24550,
 3825,
 14500,
 9900,
 6000,
 1680,
 8620,
 19950,
 7500,
 7760,
 6120,
 5760,
 6000,
 25900,
 58400,
 15950,
 126900,
 267500,
 87400,
 36450,
 75100,
 40700,
 31000,
 92700,
 3450,
 8850,
 12450,
 4815,
 37200,
 33050,
 4640,
 4105,
 7580,
 4610,
 8380,
 1650,
 21100,


In [159]:
# 크롤링 결과를 파일로 저장
CLOSINGS_LIST_DF = pd.DataFrame(CLOSINGS_LIST, columns = ['종가(2020.06.30)'])

In [ ]:
# PATH_DIR = '/content/drive/MyDrive/2021_2학기_금융/종가크롤링'
# if not os.path.exists(PATH_DIR):
#     os.makedirs(PATH_DIR
# PATH = os.path.join(PATH_DIR, '2020_06_30_크롤링_train')

In [129]:
CLOSINGS_LIST_DF.to_csv('/content/drive/MyDrive/2021_2학기_금융/2020_06_30_크롤링_train.csv', index = False)

In [163]:
CLOSINGS_LIST_DF[CLOSINGS_LIST_DF['종가(2020.06.30)'].isnull()].index

Int64Index([  696,   921,   973,  1240,  1242,  1393,  1574,  1780,  1943,
             2087,  2583,  2924,  3361,  3753,  4168,  4560,  6035,  6661,
             6824,  7171,  7679,  7927,  8197,  8198,  8199,  8200,  8201,
             8202,  8203,  8204,  8205,  8206,  8207,  8208,  8209,  8210,
             8219,  8220,  8221,  8222,  8465,  9918, 10602, 10880, 11314,
            12182, 12235, 13026, 13472, 13580, 13671, 14239, 14404, 14463,
            14835, 14915, 16055, 16278, 16822, 16834, 17067, 17537, 17707,
            17774, 18110, 18333, 20735, 21495, 21950, 22250, 23007, 23849,
            24938, 25226, 25661, 26978, 28957, 29523, 31186],
           dtype='int64')

In [177]:
MISSING_INDEX = list(CLOSINGS_LIST_DF[CLOSINGS_LIST_DF['종가(2020.06.30)'].isnull()].index)

MISSING_CODE_LIST = []
for i in MISSING_INDEX:
  MISSING_CODE_LIST.append(CODE_LIST[i])

MISSING_CODE_LIST

['033660',
 '057500',
 '057500',
 '086250',
 '204990',
 '204990',
 '023890',
 '352770',
 '028150',
 '028150',
 '057500',
 '307160',
 '293490',
 '204990',
 '057500',
 '194510',
 '033600',
 '028150',
 '057500',
 '152330',
 '152330',
 '073070',
 '340750',
 '336730',
 '334970',
 '316320',
 '314080',
 '312800',
 '300500',
 '299760',
 '266530',
 '259210',
 '258890',
 '244290',
 '239280',
 '234820',
 '142220',
 '170190',
 '177850',
 '194530',
 '057500',
 '291660',
 '033600',
 '033660',
 '057500',
 '005190',
 '033660',
 '028150',
 '204990',
 '263190',
 '028150',
 '028150',
 '003415',
 '204990',
 '028150',
 '073070',
 '043290',
 '057500',
 '152330',
 '057500',
 '028150',
 '204990',
 '204990',
 '057500',
 '028150',
 '065620',
 '023890',
 '073070',
 '003415',
 '028150',
 '028150',
 '152330',
 '057500',
 '043290',
 '043290',
 '057500',
 '057500',
 '292100',
 '028150']

In [195]:
for code in MISSING_CODE_LIST:
  CODE = code
  LAST_PAGE = CRAWL_LAST_PAGE(CODE)
  print(LAST_PAGE)

301
469
469
366
132
132
566
20
528
528
469
65
26
132
469
143
512
528
469
123
123
430
None
None
14
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
469
85
512
301
469
624
301
528
132
91
528
528
615
132
528
430
240
469
123
469
528
132
132
469
528
473
566
430
615
528
528
123
469
240
240
469
469
77
528


In [212]:
# 2020년 6월 30일 종가 추출

MISSING_CLOSINGS_LIST = []
for code in MISSING_CODE_LIST:
  CODE = code
  LAST_PAGE = CRAWL_LAST_PAGE(CODE)
  MISSING_CLOSINGS = CRAWL_CLOSINGS(2020, 6, 30, LAST_PAGE, CODE)
  MISSING_CLOSINGS_LIST.append(MISSING_CLOSINGS)

MISSING_CLOSINGS_LIST

[11150,
 4250,
 4250,
 1405,
 1225,
 1225,
 45050,
 None,
 109200,
 109200,
 4250,
 2145,
 None,
 1225,
 4250,
 536,
 179,
 109200,
 4250,
 13450,
 13450,
 492,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 4250,
 11385,
 179,
 11150,
 4250,
 16100,
 11150,
 109200,
 1225,
 53080,
 109200,
 109200,
 15200,
 1225,
 109200,
 492,
 1365,
 4250,
 13450,
 4250,
 109200,
 1225,
 1225,
 4250,
 109200,
 3190,
 45050,
 492,
 15200,
 109200,
 109200,
 13450,
 4250,
 1365,
 1365,
 4250,
 4250,
 9475,
 109200]

In [214]:
j=0
for i in MISSING_INDEX:
  CLOSINGS_LIST_DF['종가(2020.06.30)'][i] = MISSING_CLOSINGS_LIST[j]
  CLOSINGS_LIST[i] = MISSING_CLOSINGS_LIST[j]
  j += 1

CLOSINGS_LIST_DF.info()
CLOSINGS_LIST[921]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31377 entries, 0 to 31376
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   종가(2020.06.30)  31357 non-null  float64
dtypes: float64(1)
memory usage: 245.3 KB


4250

In [215]:
CLOSINGS_LIST_DF.to_csv('/content/drive/MyDrive/2021_2학기_금융/2020_06_30_크롤링_train_imputation.csv', index = False)

## **수익률 Feature 생성**

In [216]:
train['종가_0630'] = CLOSINGS_LIST

In [223]:
train

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630
0,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0
1,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A009540,2020-06-15,2020-11-16,110,1,6,0,3,2,5,5,6,9,한국조선해양,6,1,1,11,87400.0
2,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A005160,2020-06-01,2020-07-14,31,1,6,0,3,2,5,5,6,9,동국산업,10,3,99,21,2540.0
3,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,2020-05-28,2020-08-25,63,1,6,0,3,2,5,5,6,9,디와이피엔에프,10,2,99,23,9420.0
4,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,2020-05-28,2020-08-13,55,1,6,0,3,2,5,5,6,9,SNT모티브,6,2,1,23,37900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31372,4006ddaa5673365096b9dd5965e20db866f6523c162f79...,A251270,2020-06-08,2020-07-02,18,1,5,99,5,2,5,3,2,5,넷마블,5,1,1,16,100500.0
31373,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A026960,2019-07-01,2020-07-01,262,1,6,0,5,2,5,3,5,2,동서,7,1,1,261,15850.0
31374,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A012750,2020-06-08,2020-07-29,37,1,6,0,5,2,5,3,5,2,에스원,5,1,1,16,85800.0
31375,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A010130,2020-06-05,2020-07-09,24,1,6,0,5,2,5,3,5,2,고려아연,11,1,1,17,335000.0


### 잔고이력 데이터를 합치지 않아서 지금 합침 + 매수가격 Feature 생성

In [228]:
hist = pd.read_csv('/content/drive/MyDrive/2021_2학기_금융/open/stk_bnc_hist.csv')

In [229]:
# 일별 종목 잔고수량 및 금액, 액면가 정보 (2,573,839건)
# 시계열 분석을 위한 고객 별 국내 주식 잔고 이력 정보 제공
hist.rename(columns = {'act_id':'계좌ID',
                       'bse_dt':'기준일자',
                       'iem_cd':'종목코드',
                       'bnc_qty':'잔고수량',
                       'tot_aet_amt':'잔고금액',
                       'stk_par_pr':'액면가'}, inplace = True)

In [232]:
hist['매수가격'] = hist['잔고금액'] / hist['잔고수량'] # 구매 가격 feature 추가
hist = hist.fillna(0)

In [231]:
hist['기준일자']=hist['기준일자'].astype(str)
hist['기준일자']=pd.to_datetime(hist['기준일자'])

In [ ]:
hist

In [257]:
train_data = pd.merge(train, hist, how = 'left', on = ['계좌ID','종목코드'])
train_data

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격
0,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-07-10,12.0,738000.0,5000.0,61500.0
1,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-08-03,13.0,861900.0,5000.0,66300.0
2,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-09-04,0.0,0.0,5000.0,0.0
3,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-06-12,10.0,669000.0,5000.0,66900.0
4,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-06-11,8.0,548000.0,5000.0,68500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354279,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005940,2020-06-25,2020-07-01,4,1,6,0,5,2,5,3,5,2,NH투자증권,2,1,1,3,8270.0,2016-06-22,1000.0,9600000.0,5000.0,9600.0
354280,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005940,2020-06-25,2020-07-01,4,1,6,0,5,2,5,3,5,2,NH투자증권,2,1,1,3,8270.0,2016-06-07,1100.0,10461000.0,5000.0,9510.0
354281,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005940,2020-06-25,2020-07-01,4,1,6,0,5,2,5,3,5,2,NH투자증권,2,1,1,3,8270.0,2016-05-31,1200.0,10896000.0,5000.0,9080.0
354282,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005940,2020-06-25,2020-07-01,4,1,6,0,5,2,5,3,5,2,NH투자증권,2,1,1,3,8270.0,2016-05-30,1250.0,11250000.0,5000.0,9000.0


In [275]:
train_data['매수일자'] = pd.to_datetime(train_data['매수일자'])

In [277]:
train_data = train_data[train_data['매수일자'] == train_data['기준일자']]

In [278]:
train_data

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격
11,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-05-11,1.0,69400.0,5000.0,69400.0
12,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A009540,2020-06-15,2020-11-16,110,1,6,0,3,2,5,5,6,9,한국조선해양,6,1,1,11,87400.0,2020-06-15,4.0,345200.0,5000.0,86300.0
35,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A005160,2020-06-01,2020-07-14,31,1,6,0,3,2,5,5,6,9,동국산업,10,3,99,21,2540.0,2020-06-01,25.0,66125.0,1000.0,2645.0
58,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,2020-05-28,2020-08-25,63,1,6,0,3,2,5,5,6,9,디와이피엔에프,10,2,99,23,9420.0,2020-05-28,35.0,338800.0,500.0,9680.0
70,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,2020-05-28,2020-08-13,55,1,6,0,3,2,5,5,6,9,SNT모티브,6,2,1,23,37900.0,2020-05-28,2.0,77700.0,5000.0,38850.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354116,4006ddaa5673365096b9dd5965e20db866f6523c162f79...,A251270,2020-06-08,2020-07-02,18,1,5,99,5,2,5,3,2,5,넷마블,5,1,1,16,100500.0,2020-06-08,251.0,24723500.0,100.0,98500.0
354160,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A026960,2019-07-01,2020-07-01,262,1,6,0,5,2,5,3,5,2,동서,7,1,1,261,15850.0,2019-07-01,2.0,38100.0,500.0,19050.0
354200,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A012750,2020-06-08,2020-07-29,37,1,6,0,5,2,5,3,5,2,에스원,5,1,1,16,85800.0,2020-06-08,2.0,183000.0,500.0,91500.0
354224,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A010130,2020-06-05,2020-07-09,24,1,6,0,5,2,5,3,5,2,고려아연,11,1,1,17,335000.0,2020-06-05,1.0,386500.0,5000.0,386500.0


### 수익률 계산

In [285]:
train_data['수익률'] = train_data['종가_0630']/train_data['매수가격']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [286]:
train_data

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격,수익률
11,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-05-11,1.0,69400.0,5000.0,69400.0,0.904899
12,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A009540,2020-06-15,2020-11-16,110,1,6,0,3,2,5,5,6,9,한국조선해양,6,1,1,11,87400.0,2020-06-15,4.0,345200.0,5000.0,86300.0,1.012746
35,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A005160,2020-06-01,2020-07-14,31,1,6,0,3,2,5,5,6,9,동국산업,10,3,99,21,2540.0,2020-06-01,25.0,66125.0,1000.0,2645.0,0.960302
58,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,2020-05-28,2020-08-25,63,1,6,0,3,2,5,5,6,9,디와이피엔에프,10,2,99,23,9420.0,2020-05-28,35.0,338800.0,500.0,9680.0,0.973140
70,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,2020-05-28,2020-08-13,55,1,6,0,3,2,5,5,6,9,SNT모티브,6,2,1,23,37900.0,2020-05-28,2.0,77700.0,5000.0,38850.0,0.975547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354116,4006ddaa5673365096b9dd5965e20db866f6523c162f79...,A251270,2020-06-08,2020-07-02,18,1,5,99,5,2,5,3,2,5,넷마블,5,1,1,16,100500.0,2020-06-08,251.0,24723500.0,100.0,98500.0,1.020305
354160,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A026960,2019-07-01,2020-07-01,262,1,6,0,5,2,5,3,5,2,동서,7,1,1,261,15850.0,2019-07-01,2.0,38100.0,500.0,19050.0,0.832021
354200,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A012750,2020-06-08,2020-07-29,37,1,6,0,5,2,5,3,5,2,에스원,5,1,1,16,85800.0,2020-06-08,2.0,183000.0,500.0,91500.0,0.937705
354224,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A010130,2020-06-05,2020-07-09,24,1,6,0,5,2,5,3,5,2,고려아연,11,1,1,17,335000.0,2020-06-05,1.0,386500.0,5000.0,386500.0,0.866753


In [287]:
train_data.to_csv('/content/drive/MyDrive/2021_2학기_금융/train_new_feature.csv', index = False)

# **Test Data**

## 데이터 불러오기

In [7]:
cus = pd.read_csv("/content/drive/MyDrive/2021_2학기_금융/open/cus_info.csv")
iem = pd.read_csv("/content/drive/MyDrive/2021_2학기_금융/open/iem_info_20210902.csv")
hist = pd.read_csv("/content/drive/MyDrive/2021_2학기_금융/open/stk_bnc_hist.csv")
test = pd.read_csv("/content/drive/MyDrive/2021_2학기_금융/open/stk_hld_test.csv")

submission = pd.read_csv("/content/drive/MyDrive/2021_2학기_금융/open/sample_submission.csv")

### 테스트 데이터 (STK_HLD_TEST.CSV)

In [8]:
# 개발한 알고리즘 검증을 위한 문제지 (70,596건)
# 20년 12월 이전에 매수하고 21년 이후에 고객이 전량 매도한 국내주식 보유기간(일) 예측
test.rename(columns = {'act_id':'계좌ID',
                       'iem_cd':'종목코드',
                       'byn_dt':'매수일자',
                       'hist_d':'과거 보유일',
                       'submit_id':'제출ID',
                       'hold_d':'보유기간(일)'}, inplace = True)

#### 변수 설명

변수 설명
* 매수일자 : 종목 매수일자
* 과거 보유일: 매수일자 ~ 20년 12월 31일까지의 보유기간 (영업일)
* 보유기간(일) : 보유기간 (영업일) → **Target (Y)**

### 고객 정보 (CUS_INFO.CSV)

In [9]:
# 고객 및 주거래계좌 정보
# 20년 7월 이전 계좌개설 및 20년 12월 기준 국내주식 자산 100만원 이상 고객 중 10,000명 임의 추출
cus.rename(columns = {'act_id':'계좌ID',                       # 고객 (대표계좌)
                      'sex_dit_cd':'성별',                     # 1: 남성, 2: 여성
                      'cus_age_stn_cd':'연령대',               # 1: 20-24세, 2: 25-29세, 9: 60-64세
                      'ivs_icn_cd':'투자성향',
                      'cus_aet_stn_cd':'자산구간',
                      'mrz_pdt_tp_sgm_cd':'주거래상품군',
                      'lsg_sgm_cd':'Life Stage',
                      'tco_cus_grd_cd':'서비스등급',
                      'tot_ivs_te_sgm_cd':'총 투자기간',
                      'mrz_btp_dit_cd':'주거래업종구분'}, inplace = True)

#### 변수설명

변수 설명
* 계좌ID : 고객/대표계좌를 구분할 수 있는 Unique값
* 성별 : 
> - 1: 남성
> - 2 : 여성
* 연령대:
> - 1: 20 - 24세
> - 2: 25 - 29세
> - 3: 30 - 34세
> - 4: 35 - 39세
> - 5: 40 - 44세
> - 6: 45 - 49세
> - 7: 50 - 54세
> - 8: 55 - 59세
> - 9: 60 - 64세
* 투자성향:
> - 1: 안정형
> - 2: 안정추구형
> - 3: 위험중립형
> - 4: 적극투자형
> - 5: 공격투자형
> - 9: 전문투자자형
> - 0: 정보제공미동의
> - 99: 미정의 (결측치)
* 자산구간
> - 1: 0원 이상 - 1천만원 미만
> - 2: 1천만원 이상 - 3천만원 미만
> - 3: 3천만원 이상 - 5천만원 미만
> - 4: 5천만원 이상 - 1억원 미만
> - 5: 1억원 이상 - 3억원 미만
> - 6: 3억원 이상
* 주거래상품군
> - 1: Only CMA
> - 2: 국내주식
> - 3: 해외주식
> - 5: 금속
> - 6: 국내채권
> - 8: 펀드
> - 9: ELS or DLS
> - 10: 신탁_퇴직연금
> - 11: RP
> - 12: 발행어음
> - 15: 신용대출
* Life Stage
> - 2: 사회초년생
> - 3: 가족형성기(남자)
> - 4: 가족형성기(여자)
> - 5: 가족성숙기(직장인)
> - 6: 가족성숙기(주부)
> - 7: 가족성숙기(남자)
> - 8: 가족성숙기(여자)
> - 9: 은퇴기
* 서비스 등급
> - 1: 탑클래스
> - 2: 골드
> - 3: 로얄
> - 4: 그린
> - 5: 블루
> - 9: 등급 미정의
> - 99: 미정의 (결측치)
* 총 투자기간
> - 1: 6개월 미만
> - 2: 6개월 - 1년 미만
> - 3: 1년 - 3년 미만
> - 4: 3년 - 5년 미만
> - 5: 5년 - 10년 미만
> - 6: 10년 이상
* 주거래 업종 구분
> 1: 건설업
> 2: 금융업
> 3: 기계
> 4: 방송, 통신
> 5: 서비스, 오락, 문화
> 6: 운송, 운수
> 7: 유통
> 8: 의료, 의약
> 9: 전기, 전자
> 10: 제조
> 11: 철강
> 12: 화학
> 13: IT
> 14: 기타
> 15: 혼합
> 16: 비매매

### 종목 정보 (IEM_INFO.CSV)

In [10]:
# 주식종목 정보 (3,078건)
# 시계열 분석을 위한 고객 별 국내 주식 종목에 대한 코드 정보 제공

iem.rename(columns = {'iem_cd': '종목코드',
                      'iem_krl_nm': '종목한글명',
                      'btp_cfc_cd': '종목업종',
                      'mkt_pr_tal_scl_tp_cd': '시가총액 규모유형',
                      'stk_dit_cd': '시장구분'}, inplace = True)

#### 변수 설명

변수 설명
* 종목코드 : 주식종목코드
* 종목한글명 : 종목 한글명
* 종목업종 : 
> 1: 건설
> 2: 금융
> 3: 기계
> 4: 통신
> 5: 서비스
> 6: 운송
> 7: 유통
> 8: 의료
> 9: 전기
> 10: 제조
> 11: 철강
> 12: 화학
> 13: IT
> 14: 기타
* 시가총액 규모유형
> - 1: 대형주
> - 2: 중형주
> - 3: 소형주
> - 99: 기타
* 시장구분
> - 1: 코스피200
> - 2: 코스닥150
> - 99: 기타


### 국내 주식 잔고 이력 (STK_BNC_HIST.CSV)

In [11]:
# 일별 종목 잔고수량 및 금액, 액면가 정보 (2,573,839건)
# 시계열 분석을 위한 고객 별 국내 주식 잔고 이력 정보 제공
hist.rename(columns = {'act_id':'계좌ID',
                       'bse_dt':'기준일자',
                       'iem_cd':'종목코드',
                       'bnc_qty':'잔고수량',
                       'tot_aet_amt':'잔고금액',
                       'stk_par_pr':'액면가'}, inplace = True)

#### 변수설명

변수 설명
* 계좌 ID : 계좌를 구분할 수 있는 unique값
* 기준일자 : 기준일자
* 종목코드 : 거래소 종목코드
* 잔고수량 : 종목 보유 잔고수량
* 잔고금액 : 종목 평가금액
* 액면가 : 주당 액면가

## 데이터프레임 합치기

In [12]:
# 고객 정보 합치기
test_merge = pd.merge(test, cus, how = 'left', on = '계좌ID')
test_merge.head()

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8


In [13]:
# 종목 정보 합치기
test_merge = pd.merge(test_merge, iem, how = 'left', on = '종목코드')
test_merge.head()

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,파인텍,13,3,99
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2


In [14]:
# 국내 주식 잔고 이력 합치기
hist['매수가격'] = hist['잔고금액'] / hist['잔고수량'] # 매수가격 feature 추가
hist = hist.fillna(0)

In [15]:
test_merge = pd.merge(test_merge, hist, how = 'left', on = ['계좌ID','종목코드'])
test_merge = test_merge[(test_merge['매수일자'] == test_merge['기준일자'])]        # 매수일자와 기준일자가 동일한 데이터만 남김 
test_merge.reset_index(drop = True, inplace = True)

In [16]:
test_merge

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,매수가격
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1,20200522,300.0,3945000.0,5000.0,13150.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99,20190823,198.0,2524500.0,500.0,12750.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99,20200611,138.0,4291800.0,500.0,31100.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,파인텍,13,3,99,20200120,2972.0,6716720.0,500.0,2260.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2,20201217,10.0,464500.0,100.0,46450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,5,2,5,3,5,2,삼성화재,2,1,1,20201127,1.0,198500.0,500.0,198500.0
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,5,2,5,3,5,2,NH투자증권우,2,99,99,20200731,50.0,372000.0,5000.0,7440.0
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,2,2,3,4,3,8,보루네오가구,14,3,99,20170704,75057.0,3677793.0,100.0,49.0
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,2,2,3,4,3,8,종근당홀딩스,8,2,99,20201215,96.0,11424000.0,2500.0,119000.0


In [16]:
test_merge.to_csv('/content/drive/MyDrive/2021_2학기_금융/test_new.csv', index = False)

## **2020년 12월 30일 종가 크롤링**

In [75]:
test = pd.read_csv('/content/drive/MyDrive/2021_2학기_금융/test_new.csv')

In [18]:
# 종목코드 리스트 생성
CODE_LIST = []
for i in range(len(test['종목코드'])):
  CODE = test['종목코드'][i][1:]
  CODE_LIST.append(CODE)

print(CODE_LIST)

['032640', '160600', '234340', '131760', '293490', '050110', '032640', '033320', '003570', '009580', '128820', '000030', '316140', '036200', '005385', '013120', '152330', '181710', '005935', '344820', '120115', '222390', '028150', '000210', '000720', '001680', '017810', '303030', '043910', '033780', '014280', '023410', '005490', '033780', '032640', '035420', '005930', '005940', '008770', '011930', '103590', '069500', '101530', '114090', '041460', '003380', '083310', '100130', '200130', '122350', '086280', '024660', '038070', '028100', '065650', '123010', '174900', '027740', '064350', '033230', '293490', '087010', '086040', '065660', '043610', '142760', '008290', '035720', '326030', '049180', '041190', '121800', '187220', '042660', '220260', '003010', '004270', '096610', '248170', '263770', '092220', '015760', '001120', '161890', '042500', '252670', '004780', '092130', '009580', '001630', '035900', '039830', '033660', '215000', '022100', '035810', '288620', '161390', '194700', '148150',

In [19]:
len(CODE_LIST)

70596

In [ ]:
UNIQUE_CODE_LIST = []

for code in CODE_LIST: 
  if code not in UNIQUE_CODE_LIST: 
    UNIQUE_CODE_LIST.append(code) 

UNIQUE_CODE_LIST

In [21]:
len(UNIQUE_CODE_LIST)

2708

In [ ]:
# 2020년 12월 30일 종가 추출

UNIQUE_CLOSINGS_LIST = []
for code in UNIQUE_CODE_LIST:
  CODE = code
  PAGE = 18                                                         # 2020-12-30은 대개 페이지 18에 존재
  CLOSINGS = CRAWL_20200630_CLOSINGS(2020, 12, 30, CODE, PAGE)
  UNIQUE_CLOSINGS_LIST.append(CLOSINGS)

UNIQUE_CLOSINGS_LIST

In [25]:
# 2020년 12월 30일 종가 크롤링 결과를 csv 파일로 저장
UNIQUE_CLOSINGS_LIST_DF = pd.DataFrame(UNIQUE_CLOSINGS_LIST, columns = ['종가(2020.12.30)'])

In [ ]:
# PATH_DIR = '/content/drive/MyDrive/2021_2학기_금융/종가크롤링'
# if not os.path.exists(PATH_DIR):
#     os.makedirs(PATH_DIR)
# PATH = os.path.join(PATH_DIR, '2020_12_30_크롤링_test')

In [26]:
UNIQUE_CLOSINGS_LIST_DF.to_csv('/content/drive/MyDrive/2021_2학기_금융/2020_12_30_크롤링_test_unique.csv', index = False)

In [47]:
UNIQUE_CLOSINGS_LIST_DF = pd.read_csv('/content/drive/MyDrive/2021_2학기_금융/2020_12_30_크롤링_test_unique.csv')

### 처리 가능한 결측치 처리 (18 페이지에 2020년 12월 30일 종가가 없는 경우)

In [48]:
UNIQUE_CLOSINGS_LIST_DF[UNIQUE_CLOSINGS_LIST_DF['종가(2020.12.30)'].isnull()].index

Int64Index([  10,   15,   20,   21,   48,   87,  176,  177,  238,  247,
            ...
            2676, 2678, 2679, 2681, 2687, 2696, 2698, 2703, 2705, 2707],
           dtype='int64', length=212)

In [ ]:
UNIQUE_MISSING_INDEX = list(UNIQUE_CLOSINGS_LIST_DF[UNIQUE_CLOSINGS_LIST_DF['종가(2020.12.30)'].isnull()].index)

UNIQUE_MISSING_CODE_LIST = []
for i in UNIQUE_MISSING_INDEX:
  UNIQUE_MISSING_CODE_LIST.append(UNIQUE_CODE_LIST[i])

UNIQUE_MISSING_CODE_LIST

In [48]:
# # 코드 확인
# last_list = []
# for code in test['종목코드']:
#   if len(code) != 7:
#     print(code)
#   if 'A' not in code:
#     print(code)
#   if code[-1] not in ['0','1', '2', '3', '4', '5', '6', '7', '8', '9']:
#     last = code[-1]
#     if last not in last_list:
#       last_list.append(last)

# last_list   # K, L, Q, N  

['K', 'L', 'Q', 'N']

In [28]:
# for code in test['종목코드']:
#   if code[-1] == 'Q' or code[-1] == 'N':
#     print(test['종목한글명'][test['종목코드']==code])

3269    발레앤모델6우
Name: 종목한글명, dtype: object
22388    체인파트너스4우
Name: 종목한글명, dtype: object


In [52]:
# 2020년 12월 30일 종가 추출

UNIQUE_MISSING_CLOSINGS_LIST = []
for code in UNIQUE_MISSING_CODE_LIST:
  CODE = code
  LAST_PAGE = CRAWL_LAST_PAGE(CODE)
  UNIQUE_MISSING_CLOSINGS = CRAWL_CLOSINGS(2020, 12, 30, LAST_PAGE, CODE)
  UNIQUE_MISSING_CLOSINGS_LIST.append(UNIQUE_MISSING_CLOSINGS)

UNIQUE_MISSING_CLOSINGS_LIST

[None,
 None,
 None,
 139000,
 None,
 12200,
 1405,
 None,
 None,
 862,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 407,
 None,
 None,
 2075,
 1100,
 None,
 None,
 None,
 None,
 279,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 56500,
 None,
 None,
 None,
 None,
 None,
 214,
 None,
 None,
 None,
 None,
 None,
 None,
 2220,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 1770,
 None,
 None,
 None,
 179,
 None,
 None,
 2520,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 15950,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [53]:
j=0
for i in UNIQUE_MISSING_INDEX:
  UNIQUE_CLOSINGS_LIST_DF['종가(2020.12.30)'][i] = UNIQUE_MISSING_CLOSINGS_LIST[j]
  # UNIQUE_CLOSINGS_LIST[i] = UNIQUE_MISSING_CLOSINGS_LIST[j]
  j += 1

UNIQUE_CLOSINGS_LIST_DF

,종가(2020.12.30)
0,11750.0
1,310.0
2,33150.0
3,1705.0
4,46000.0
...,...
2703,NaN
2704,9465.0
2705,NaN
2706,2630.0


In [100]:
UNIQUE_CLOSINGS_LIST_DF.to_csv('/content/drive/MyDrive/2021_2학기_금융/2020_12_30_크롤링_test_unique_imputation.csv', index = False)

## **수익률 Feature 생성**

In [65]:
UNIQUE_CLOSINGS_LIST_DF['종목코드'] = UNIQUE_CODE_LIST

In [66]:
print(len(test))
print(len(UNIQUE_CLOSINGS_LIST_DF))

70596
2708


In [67]:
UNIQUE_CLOSINGS_LIST_DF

,종가(2020.12.30),종목코드
0,11750.0,032640
1,310.0,160600
2,33150.0,234340
3,1705.0,131760
4,46000.0,293490
...,...,...
2703,NaN,153760
2704,9465.0,217770
2705,NaN,35732K
2706,2630.0,182690


In [68]:
for i in range(len(UNIQUE_CLOSINGS_LIST_DF)):
  UNIQUE_CLOSINGS_LIST_DF['종목코드'][i] = 'A' + UNIQUE_CLOSINGS_LIST_DF['종목코드'][i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
UNIQUE_CLOSINGS_LIST_DF

,종가(2020.12.30),종목코드
0,11750.0,A032640
1,310.0,A160600
2,33150.0,A234340
3,1705.0,A131760
4,46000.0,A293490
...,...,...
2703,NaN,A153760
2704,9465.0,A217770
2705,NaN,A35732K
2706,2630.0,A182690


In [73]:
UNIQUE_CLOSINGS_LIST_DF = UNIQUE_CLOSINGS_LIST_DF.rename(columns = {'종가(2020.12.30)':'종가_1230'})

In [74]:
UNIQUE_CLOSINGS_LIST_DF

,종가_1230,종목코드
0,11750.0,A032640
1,310.0,A160600
2,33150.0,A234340
3,1705.0,A131760
4,46000.0,A293490
...,...,...
2703,NaN,A153760
2704,9465.0,A217770
2705,NaN,A35732K
2706,2630.0,A182690


In [101]:
UNIQUE_CLOSINGS_LIST_DF.to_csv('/content/drive/MyDrive/2021_2학기_금융/종가크롤링/2020_12_30_크롤링_test_unique_imputation_with_종목코드.csv', index = False)

In [76]:
# test 데이터에 2020년 12월 30일 종가 피쳐 추가
test_merge = pd.merge(test, UNIQUE_CLOSINGS_LIST_DF, on = '종목코드', how = 'left')

In [77]:
test_merge

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,매수가격,종가_1230
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1,20200522,300.0,3945000.0,5000.0,13150.0,11750.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99,20190823,198.0,2524500.0,500.0,12750.0,310.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99,20200611,138.0,4291800.0,500.0,31100.0,33150.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,파인텍,13,3,99,20200120,2972.0,6716720.0,500.0,2260.0,1705.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2,20201217,10.0,464500.0,100.0,46450.0,46000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,5,2,5,3,5,2,삼성화재,2,1,1,20201127,1.0,198500.0,500.0,198500.0,187500.0
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,5,2,5,3,5,2,NH투자증권우,2,99,99,20200731,50.0,372000.0,5000.0,7440.0,8760.0
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,2,2,3,4,3,8,보루네오가구,14,3,99,20170704,75057.0,3677793.0,100.0,49.0,NaN
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,2,2,3,4,3,8,종근당홀딩스,8,2,99,20201215,96.0,11424000.0,2500.0,119000.0,127500.0


In [79]:
test_merge['종가_1230'].isnull().sum()

549

In [80]:
test_merge['종가_1230']

0         11750.0
1           310.0
2         33150.0
3          1705.0
4         46000.0
           ...   
70591    187500.0
70592      8760.0
70593         NaN
70594    127500.0
70595    226500.0
Name: 종가_1230, Length: 70596, dtype: float64

In [81]:
test_merge['매수가격']

0         13150.0
1         12750.0
2         31100.0
3          2260.0
4         46450.0
           ...   
70591    198500.0
70592      7440.0
70593        49.0
70594    119000.0
70595    228000.0
Name: 매수가격, Length: 70596, dtype: float64

In [91]:
test_merge[test_merge['종가_1230'].isnull()]

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,매수가격,종가_1230
11,64aae8dd71e5c0761000db9f9a6779e504e7a4aa9dc097...,A000030,20160104,1227,IDX00012,0,1,4,99,1,2,3,3,6,16,우리은행,14,1,1,20160104,340.0,2924000.0,5000.0,8600.0,NaN
16,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,A152330,20200720,112,IDX00017,0,1,6,4,4,2,5,2,6,1,코리아오토글라스,14,3,99,20200720,100.0,1480000.0,5000.0,14800.0,NaN
21,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,A222390,20160104,1227,IDX00022,0,1,6,4,4,2,5,2,6,1,케이비제8호스팩,14,3,99,20160104,2481.0,4962000.0,100.0,2000.0,NaN
51,6d497facfa1ea5901b827335553331f8555fec02a8184f...,A024660,20160715,1096,IDX00052,0,2,6,4,4,8,5,3,6,16,하림홀딩스,14,2,99,20160715,200.0,1100000.0,500.0,5500.0,NaN
125,66d8611661423c1e0764994be4a7afe772e70a8f7f3cd2...,A000030,20180207,712,IDX00126,0,1,9,4,4,2,9,4,6,16,우리은행,14,1,1,20180207,1000.0,15600000.0,5000.0,15600.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66795,720d6550786a3470dcbf15cce60007803e7bc537a7acb0...,A134780,20200626,128,IDX66796,0,1,1,99,2,2,2,5,2,10,화진,14,3,99,20200626,1515.0,757500.0,500.0,500.0,NaN
67678,128a2e745f292c14bebb7edff841caa6170a9b1cb122ea...,A112240,20200625,129,IDX67679,0,1,4,3,1,2,3,9,1,13,에스에프씨,14,3,99,20200625,211.0,27430.0,500.0,130.0,NaN
68702,31cffdb69fe18ca06cd0044007f8793e18c6d5d28dfeeb...,A35732K,20200724,108,IDX68703,0,1,4,5,5,2,3,4,2,13,하스퍼1우,14,99,99,20200724,20.0,100000.0,5000.0,5000.0,NaN
70414,94137a989b8c4b3398805fbc7a5690a31a2b199cd006c2...,A003415,20201111,35,IDX70415,0,1,9,2,1,2,9,5,2,10,쌍용양회우,14,99,99,20201111,90.0,2281500.0,1000.0,25350.0,NaN


In [93]:
# 수익률 계산
EARNING_LIST = []
for i in range(len(test_merge)):
  earning = (test_merge['종가_1230'][i] / test_merge['매수가격'][i]) - 1  # float인지 확인. 분모가 0인지 확인

  EARNING_LIST.append(earning)

EARNING_LIST

[-0.10646387832699622,
 -0.9756862745098039,
 0.06591639871382626,
 -0.24557522123893805,
 -0.009687836383207804,
 -0.1027287319422151,
 0.03524229074889873,
 -0.14341463414634148,
 0.09888059701492535,
 -0.46370370370370373,
 -0.1416781292984869,
 nan,
 -0.3640522875816994,
 0.05541871921182273,
 0.007972665148063829,
 0.0379487179487179,
 nan,
 -0.023560209424083767,
 0.7378984651711924,
 -0.1046228710462287,
 0.013513513513513598,
 nan,
 -0.2052601486563751,
 0.02091020910209096,
 0.03310344827586209,
 -0.0019157088122605526,
 0.009146341463414531,
 -0.24622356495468278,
 -0.41151385927505335,
 -0.1648241206030151,
 0.3425,
 0.10023310023310028,
 -0.07482993197278909,
 0.05456852791878175,
 -0.004237288135593209,
 -0.059485530546623755,
 0.5758754863813229,
 0.17831074035453587,
 0.18417266187050352,
 0.4545454545454546,
 -0.36978131212723653,
 0.38353765323992994,
 -0.7980416156670747,
 -0.2863247863247863,
 -0.1877394636015326,
 -0.32509363295880145,
 0.13095238095238093,
 0.74214

In [94]:
# test 데이터에 수익률 피쳐 추가
test_merge['수익률'] = EARNING_LIST

In [96]:
test_merge

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,매수가격,종가_1230,수익률
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,2,9,5,5,8,LG유플러스,4,1,1,20200522,300.0,3945000.0,5000.0,13150.0,11750.0,-0.106464
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,2,9,5,5,8,이큐셀,10,3,99,20190823,198.0,2524500.0,500.0,12750.0,310.0,-0.975686
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,2,9,5,5,8,세틀뱅크,13,2,99,20200611,138.0,4291800.0,500.0,31100.0,33150.0,0.065916
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,2,9,5,5,8,파인텍,13,3,99,20200120,2972.0,6716720.0,500.0,2260.0,1705.0,-0.245575
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,2,9,5,5,8,카카오게임즈,13,1,2,20201217,10.0,464500.0,100.0,46450.0,46000.0,-0.009688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,5,2,5,3,5,2,삼성화재,2,1,1,20201127,1.0,198500.0,500.0,198500.0,187500.0,-0.055416
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,5,2,5,3,5,2,NH투자증권우,2,99,99,20200731,50.0,372000.0,5000.0,7440.0,8760.0,0.177419
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,2,2,3,4,3,8,보루네오가구,14,3,99,20170704,75057.0,3677793.0,100.0,49.0,NaN,NaN
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,2,2,3,4,3,8,종근당홀딩스,8,2,99,20201215,96.0,11424000.0,2500.0,119000.0,127500.0,0.071429


In [95]:
test_merge[test_merge['수익률'].isnull()]

,계좌ID,종목코드,매수일자,과거 보유일,제출ID,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,기준일자,잔고수량,잔고금액,액면가,매수가격,종가_1230,수익률
11,64aae8dd71e5c0761000db9f9a6779e504e7a4aa9dc097...,A000030,20160104,1227,IDX00012,0,1,4,99,1,2,3,3,6,16,우리은행,14,1,1,20160104,340.0,2924000.0,5000.0,8600.0,NaN,NaN
16,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,A152330,20200720,112,IDX00017,0,1,6,4,4,2,5,2,6,1,코리아오토글라스,14,3,99,20200720,100.0,1480000.0,5000.0,14800.0,NaN,NaN
21,5f7c3a8f37d9c268d06130ff0be5d32a1b9ef68c13049f...,A222390,20160104,1227,IDX00022,0,1,6,4,4,2,5,2,6,1,케이비제8호스팩,14,3,99,20160104,2481.0,4962000.0,100.0,2000.0,NaN,NaN
51,6d497facfa1ea5901b827335553331f8555fec02a8184f...,A024660,20160715,1096,IDX00052,0,2,6,4,4,8,5,3,6,16,하림홀딩스,14,2,99,20160715,200.0,1100000.0,500.0,5500.0,NaN,NaN
125,66d8611661423c1e0764994be4a7afe772e70a8f7f3cd2...,A000030,20180207,712,IDX00126,0,1,9,4,4,2,9,4,6,16,우리은행,14,1,1,20180207,1000.0,15600000.0,5000.0,15600.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66795,720d6550786a3470dcbf15cce60007803e7bc537a7acb0...,A134780,20200626,128,IDX66796,0,1,1,99,2,2,2,5,2,10,화진,14,3,99,20200626,1515.0,757500.0,500.0,500.0,NaN,NaN
67678,128a2e745f292c14bebb7edff841caa6170a9b1cb122ea...,A112240,20200625,129,IDX67679,0,1,4,3,1,2,3,9,1,13,에스에프씨,14,3,99,20200625,211.0,27430.0,500.0,130.0,NaN,NaN
68702,31cffdb69fe18ca06cd0044007f8793e18c6d5d28dfeeb...,A35732K,20200724,108,IDX68703,0,1,4,5,5,2,3,4,2,13,하스퍼1우,14,99,99,20200724,20.0,100000.0,5000.0,5000.0,NaN,NaN
70414,94137a989b8c4b3398805fbc7a5690a31a2b199cd006c2...,A003415,20201111,35,IDX70415,0,1,9,2,1,2,9,5,2,10,쌍용양회우,14,99,99,20201111,90.0,2281500.0,1000.0,25350.0,NaN,NaN


In [97]:
test_merge.to_csv('/content/drive/MyDrive/2021_2학기_금융/test_new_feature.csv', index = False)

## (별도) Train Data 수익률 - 1

In [111]:
train = pd.read_csv('/content/drive/Shareddrives/주식 보유기간 예측 (NH투자증권 빅데이터 경진대회)/장하연/DATA_수익률_feature_추가/train_new_feature.csv')

In [112]:
train

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격,수익률
0,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-05-11,1.0,69400.0,5000.0,69400.0,0.904899
1,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A009540,2020-06-15,2020-11-16,110,1,6,0,3,2,5,5,6,9,한국조선해양,6,1,1,11,87400.0,2020-06-15,4.0,345200.0,5000.0,86300.0,1.012746
2,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A005160,2020-06-01,2020-07-14,31,1,6,0,3,2,5,5,6,9,동국산업,10,3,99,21,2540.0,2020-06-01,25.0,66125.0,1000.0,2645.0,0.960302
3,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,2020-05-28,2020-08-25,63,1,6,0,3,2,5,5,6,9,디와이피엔에프,10,2,99,23,9420.0,2020-05-28,35.0,338800.0,500.0,9680.0,0.973140
4,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,2020-05-28,2020-08-13,55,1,6,0,3,2,5,5,6,9,SNT모티브,6,2,1,23,37900.0,2020-05-28,2.0,77700.0,5000.0,38850.0,0.975547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31372,4006ddaa5673365096b9dd5965e20db866f6523c162f79...,A251270,2020-06-08,2020-07-02,18,1,5,99,5,2,5,3,2,5,넷마블,5,1,1,16,100500.0,2020-06-08,251.0,24723500.0,100.0,98500.0,1.020305
31373,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A026960,2019-07-01,2020-07-01,262,1,6,0,5,2,5,3,5,2,동서,7,1,1,261,15850.0,2019-07-01,2.0,38100.0,500.0,19050.0,0.832021
31374,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A012750,2020-06-08,2020-07-29,37,1,6,0,5,2,5,3,5,2,에스원,5,1,1,16,85800.0,2020-06-08,2.0,183000.0,500.0,91500.0,0.937705
31375,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A010130,2020-06-05,2020-07-09,24,1,6,0,5,2,5,3,5,2,고려아연,11,1,1,17,335000.0,2020-06-05,1.0,386500.0,5000.0,386500.0,0.866753


In [113]:
train['수익률']

0        0.904899
1        1.012746
2        0.960302
3        0.973140
4        0.975547
           ...   
31372    1.020305
31373    0.832021
31374    0.937705
31375    0.866753
31376    0.979858
Name: 수익률, Length: 31377, dtype: float64

In [114]:
train['수익률'] = train['수익률'] - 1

In [115]:
train

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격,수익률
0,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A004800,2020-05-11,2020-09-03,83,1,6,0,3,2,5,5,6,9,효성,2,2,1,36,62800.0,2020-05-11,1.0,69400.0,5000.0,69400.0,-0.095101
1,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A009540,2020-06-15,2020-11-16,110,1,6,0,3,2,5,5,6,9,한국조선해양,6,1,1,11,87400.0,2020-06-15,4.0,345200.0,5000.0,86300.0,0.012746
2,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A005160,2020-06-01,2020-07-14,31,1,6,0,3,2,5,5,6,9,동국산업,10,3,99,21,2540.0,2020-06-01,25.0,66125.0,1000.0,2645.0,-0.039698
3,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A104460,2020-05-28,2020-08-25,63,1,6,0,3,2,5,5,6,9,디와이피엔에프,10,2,99,23,9420.0,2020-05-28,35.0,338800.0,500.0,9680.0,-0.026860
4,b6623032ed365b6c9a71c0344ebb8ca191f1ebc5927771...,A064960,2020-05-28,2020-08-13,55,1,6,0,3,2,5,5,6,9,SNT모티브,6,2,1,23,37900.0,2020-05-28,2.0,77700.0,5000.0,38850.0,-0.024453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31372,4006ddaa5673365096b9dd5965e20db866f6523c162f79...,A251270,2020-06-08,2020-07-02,18,1,5,99,5,2,5,3,2,5,넷마블,5,1,1,16,100500.0,2020-06-08,251.0,24723500.0,100.0,98500.0,0.020305
31373,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A026960,2019-07-01,2020-07-01,262,1,6,0,5,2,5,3,5,2,동서,7,1,1,261,15850.0,2019-07-01,2.0,38100.0,500.0,19050.0,-0.167979
31374,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A012750,2020-06-08,2020-07-29,37,1,6,0,5,2,5,3,5,2,에스원,5,1,1,16,85800.0,2020-06-08,2.0,183000.0,500.0,91500.0,-0.062295
31375,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A010130,2020-06-05,2020-07-09,24,1,6,0,5,2,5,3,5,2,고려아연,11,1,1,17,335000.0,2020-06-05,1.0,386500.0,5000.0,386500.0,-0.133247


In [116]:
train.to_csv('/content/drive/Shareddrives/주식 보유기간 예측 (NH투자증권 빅데이터 경진대회)/장하연/DATA_수익률_feature_추가/train_new_feature.csv', index = False)

In [117]:
train[train['수익률'].isnull()]

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격,수익률
1780,a2c23e8c51658294dda72811c7dbe9a05a72aa6c646734...,A352770,2020-06-05,2020-11-30,126,1,3,3,3,2,3,4,4,5,클리노믹스,5,99,99,17,NaN,2020-06-05,1000.0,100000.0,100.0,100.0,NaN
3361,26f7f15a177e8104a2d7e2d4112df5669aaa88ddc17602...,A293490,2018-05-24,2020-08-04,573,2,6,99,3,2,8,4,4,16,카카오게임즈,13,1,2,548,NaN,2018-05-24,200.0,20000.0,100.0,100.0,NaN
8197,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A340750,2019-12-17,2020-09-15,195,1,7,4,5,2,7,2,3,10,씨엘바이오,14,99,99,140,NaN,2019-12-17,1000.0,500000.0,500.0,500.0,NaN
8198,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A336730,2020-01-07,2020-09-18,183,1,7,4,5,2,7,2,3,10,비욘드바이오,14,99,99,125,NaN,2020-01-07,50.0,25000.0,500.0,500.0,NaN
8199,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A334970,2019-10-15,2020-09-15,240,1,7,4,5,2,7,2,3,10,프레스티지바이오로직스,14,99,99,185,NaN,2019-10-15,1800.0,900000.0,500.0,500.0,NaN
8200,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A316320,2019-01-14,2020-08-28,424,1,7,4,5,2,7,2,3,10,메티메디제약,14,99,99,381,NaN,2019-01-14,1240.0,620000.0,500.0,500.0,NaN
8201,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A314080,2019-06-26,2020-09-08,314,1,7,4,5,2,7,2,3,10,이노핀,14,99,99,264,NaN,2019-06-26,800.0,80000.0,100.0,100.0,NaN
8202,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A312800,2019-01-17,2020-08-28,421,1,7,4,5,2,7,2,3,10,퓨젠바이오,14,99,99,378,NaN,2019-01-17,500.0,250000.0,500.0,500.0,NaN
8203,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A300500,2018-07-06,2020-08-17,551,1,7,4,5,2,7,2,3,10,에이디엠,14,99,99,517,NaN,2018-07-06,1000.0,5000000.0,5000.0,5000.0,NaN
8204,5ba2b02c3eecba6c6a2127c2d830a5777fc82ca097e63d...,A299760,2018-09-04,2020-08-18,510,1,7,4,5,2,7,2,3,10,더코더,14,99,99,475,NaN,2018-09-04,500.0,250000.0,500.0,500.0,NaN


In [118]:
train[train['수익률'] == 0]

,계좌ID,종목코드,매수일자,매도일자,보유기간(일),성별,연령대,투자성향,자산구간,주거래상품군,Life Stage,서비스등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분,hist_d,종가_0630,기준일자,잔고수량,잔고금액,액면가,매수가격,수익률
11,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,A035720,2020-06-29,2020-09-02,47,2,7,4,3,2,5,5,6,9,카카오,5,1,1,1,267500.0,2020-06-29,20.0,5350000.0,500.0,267500.0,0.0
57,2c17850345fbac9b3f004773972fd72a716228b3896e6f...,A080720,2020-06-30,2020-07-23,17,2,4,3,4,2,4,3,5,10,한국유니온제약,10,3,99,0,17900.0,2020-06-30,10.0,179000.0,500.0,17900.0,0.0
65,10b0ce16d9099893611dda555bd60ff0eeb3010f092a61...,A000240,2020-06-30,2020-07-01,1,1,9,2,1,2,9,5,6,5,한국앤컴퍼니,2,2,1,0,12950.0,2020-06-30,861.0,11149950.0,500.0,12950.0,0.0
78,dbc7f1c8694e7a40827680f9d8b8538db774a134fa24bb...,A237690,2020-06-30,2020-07-01,1,1,7,3,3,2,5,3,5,2,에스티팜,10,1,2,0,40300.0,2020-06-30,200.0,8060000.0,500.0,40300.0,0.0
81,dbc7f1c8694e7a40827680f9d8b8538db774a134fa24bb...,A060900,2020-06-30,2020-07-01,1,1,7,3,3,2,5,3,5,2,대한그린파워,10,3,99,0,1195.0,2020-06-30,14510.0,17339450.0,100.0,1195.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31359,68e6fe3b7d7008a4ee598448ec139d09e274d1a2b618d2...,A048410,2020-06-30,2020-07-01,1,2,4,3,1,2,4,5,2,10,현대바이오,10,1,2,0,13250.0,2020-06-30,60.0,795000.0,500.0,13250.0,0.0
31360,68e6fe3b7d7008a4ee598448ec139d09e274d1a2b618d2...,A047820,2020-06-30,2020-07-01,1,2,4,3,1,2,4,5,2,10,초록뱀미디어,5,2,99,0,1660.0,2020-06-30,2232.0,3705120.0,500.0,1660.0,0.0
31361,5919f1968b57e9789dd35f36c2ae607a76f8dbb1dcf85c...,A005935,2020-06-30,2020-09-08,50,2,7,4,1,2,8,9,2,9,삼성전자우,9,99,99,0,46500.0,2020-06-30,2.0,93000.0,100.0,46500.0,0.0
31362,068751c4964fc225a8e0fae3ce7cfb4aeb1fdc8ba82443...,A041140,2020-06-30,2020-07-02,2,1,6,99,2,2,5,4,2,2,넥슨지티,13,2,99,0,12200.0,2020-06-30,100.0,1220000.0,500.0,12200.0,0.0
